<image src="https://i.imgur.com/r2668pf.png" width=100%>

## <font color='#4C5FDA'>**Instalamos paquetes y librerias**</font>


In [46]:
%pip install -q mat73
%pip install -q openexr
%pip install -q spectral
%pip install -q scikit-image
%pip install -q gdown

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [47]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Scientific computing
import numpy as np
from scipy import io
from scipy import interpolate
from scipy.signal import gaussian

# Plotting
import matplotlib.pyplot as plt
plt.ion()

from sklearn.decomposition import PCA

## <font color='#4C5FDA'>**Descargamos la imagen hiperspectral y la libreria de firmas espectrales**</font>


In [ ]:
!gdown 1L78cLVSSFLgvgGHWjGlaCPG4MzqPhYBz
!gdown 1F8qD2vwjIkFHcHwkXsVzBAPAXyH9DhyZ

## <font color='#4C5FDA'>**Cargamos el cubo espectral**</font>


In [48]:
file = mat73.loadmat('./8.mat')
cube = file["rad"]

cube = cube / np.max(cube)
cube = cube.transpose(1,0 ,2)[::-1, :, :]

In [ ]:
plt.imshow(cube[:,:,0])

## <font color='#4C5FDA'>**Metodos utiles para transformar una imagen hiperspectral a rgb**</font>


In [50]:
import numpy as np


def g(x, alpha, mu, sigma1, sigma2):
    sigma = (x < mu)*sigma1 + (x >= mu)*sigma2
    return alpha*np.exp((x-mu)**2 / (-2*(sigma**2)))


def component_x(x): return g(x, 1.056, 5998, 379, 310) + \
    g(x, 0.362, 4420, 160, 267) + g(x, -0.065, 5011, 204, 262)


def component_y(x): return g(x, 0.821, 5688, 469, 405) + \
    g(x, 0.286, 5309, 163, 311)


def component_z(x): return g(x, 1.217, 4370, 118, 360) + \
    g(x, 0.681, 4590, 260, 138)


def xyz_from_xy(x, y):
    """Return the vector (x, y, 1-x-y)."""
    return np.array((x, y, 1-x-y))


ILUMINANT = {
    'D65': xyz_from_xy(0.3127, 0.3291),
    'E':  xyz_from_xy(1/3, 1/3),
}

COLOR_SPACE = {
    'sRGB': (xyz_from_xy(0.64, 0.33),
             xyz_from_xy(0.30, 0.60),
             xyz_from_xy(0.15, 0.06),
             ILUMINANT['D65']),

    'AdobeRGB': (xyz_from_xy(0.64, 0.33),
                 xyz_from_xy(0.21, 0.71),
                 xyz_from_xy(0.15, 0.06),
                 ILUMINANT['D65']),

    'AppleRGB': (xyz_from_xy(0.625, 0.34),
                 xyz_from_xy(0.28, 0.595),
                 xyz_from_xy(0.155, 0.07),
                 ILUMINANT['D65']),

    'UHDTV': (xyz_from_xy(0.708, 0.292),
              xyz_from_xy(0.170, 0.797),
              xyz_from_xy(0.131, 0.046),
              ILUMINANT['D65']),

    'CIERGB': (xyz_from_xy(0.7347, 0.2653),
               xyz_from_xy(0.2738, 0.7174),
               xyz_from_xy(0.1666, 0.0089),
               ILUMINANT['E']),
}


class ColourSystem:

    def __init__(self, start=None, end=None, num=None, cs='sRGB'):

        # Chromaticities
        bands = np.linspace(start=start, stop=end, num=num)*10

        self.cmf = np.array([component_x(bands),
                             component_y(bands),
                             component_z(bands)])

        self.red, self.green, self.blue, self.white = COLOR_SPACE[cs]

        # The chromaticity matrix (rgb -> xyz) and its inverse
        self.M = np.vstack((self.red, self.green, self.blue)).T
        self.MI = np.linalg.inv(self.M)

        # White scaling array
        self.wscale = self.MI.dot(self.white)

        # xyz -> rgb transformation matrix
        self.A = self.MI / self.wscale[:, np.newaxis]

  
    def get_transform_matrix(self):

        XYZ = self.cmf
        RGB = XYZ.T @ self.A.T
        RGB = RGB / np.sum(RGB, axis=0, keepdims=True)
        return RGB

    def spec_to_rgb(self, spec):
        """Convert a spectrum to an rgb value."""
        M = self.get_transform_matrix()
        rgb = spec @ M
        return rgb

# <font color='#4C5FDA'>**pasamos de 31 bandas a 3 bandas**</font>


In [ ]:
color_space = "sRGB"
start, end = None, None
number_bands = None

cs = ColourSystem(cs=color_space, start=start, end=end, num=number_bands)

img_rgb = cs.spec_to_rgb(cube)

img_rgb = np.clip(img_rgb, 0, 1)

#post processing, because the image is too dark
img_rgb = img_rgb ** 0.5

plt.imshow(img_rgb)
plt.colorbar()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(img_rgb)
axs[0].set_title('RGB image')

axs[1].imshow(cube[:,:,10])
axs[1].set_title('Spectral image: band number 10')

plt.show()



# <font color='#4C5FDA'>**Cargamos la libreria de firmas espectrales**</font>


In [53]:
import pandas
import numpy as np

spectraldb = pandas.read_csv('spectraldb.csv')
print(spectraldb.columns)
print(spectraldb.shape)

Index(['ID', 'Name', 'ObjectType', 'RadianceType', 'MeasurementLocation',
       'Lat', 'Long', 'MeasurementCredit', 'URL', 'Comments', 'Specularity',
       'Roughness', 'MeasurementType', 'R', 'G', 'B', 'L', 'a', 'b', 'hex',
       'SCIMeasures', 'SCEMeasures', 'VisibleReflectance',
       'MelanopicReflectance', 'Radiance Material'],
      dtype='object')
(1288, 25)


# <font color='#4C5FDA'>**Pre-procesamos las firmas**</font>


In [54]:
spectraldb = spectraldb[spectraldb['ObjectType'] != 'Color Swatch']
print(spectraldb.shape)

(1080, 25)


In [ ]:
materials = ["plant", "stone", "Aluminium", "Concrete", "Wooden", "Wood", "Metal", "Fabric", "Metallic", "grass"]

materials_dict = {word.lower(): index for index, word in enumerate(materials)}

categories = ["Wall", "Plant"]

materials_dict["wall"] = len(materials_dict)
materials_dict["plant"] = len(materials_dict)

# search for material with name that contains some string from materials list

filter1 = spectraldb["ObjectType"].str.contains('|'.join(categories), case=False)
wall_plant = spectraldb[filter1]

filter = spectraldb["Name"].str.contains('|'.join(materials), case=False)
spectraldb = spectraldb[filter]

#join two dataframes, maybe are the same row in both dataframes so we need to drop duplicates

spectraldb = pandas.concat([spectraldb, wall_plant]).drop_duplicates().reset_index(drop=True)

spectraldb

In [56]:
wv_int = np.array(list(range(None, None, None))) # desde la longitud de onda 420 a la 730 cada 10 nm
wv_int.shape

(31,)

## <font color='#4C5FDA'>**Plot de todas las firmas espectrales de materiales**</font>


In [ ]:
data = []


remove_signatures = []

for row in spectraldb.itertuples():
    SCEdict = eval(row.SCEMeasures)
    try:
        reflectances = np.array([SCEdict[wavelength] for wavelength in wv_int]) / 100
    except KeyError:
        remove_signatures.append(row.Index)
        continue
    #if row.ObjectType not in objects.keys():
    #    objects[row.ObjectType] = len(objects)
        
    data.append(reflectances)
    
    plt.plot(wv_int, reflectances)

spectraldb.drop(remove_signatures, inplace=True)

print(np.array(data).shape)

# <font color='#4C5FDA'>**Plot de firmas separado por material**</font>


In [ ]:

fig, ax = plt.subplots(3, 4, figsize=(10, 5))

for i,material in enumerate(materials_dict.keys()):
    filter = spectraldb["Name"].str.contains(material, case=False)
    material_df = spectraldb[filter]

    filter1 = spectraldb["ObjectType"].str.contains(material, case=False)
    material_df_ = spectraldb[filter1]

    signatures_mat = pandas.concat([material_df, material_df_]).drop_duplicates().reset_index()

    print(material, signatures_mat.shape[0])

    for row in signatures_mat.itertuples():
        SCEdict = eval(row.SCEMeasures)
        reflectances = np.array([SCEdict[wavelength] for wavelength in wv_int]) / 100
        ax[i//4, i%4].plot(wv_int, reflectances)
        ax[i//4, i%4].set_title(material)
        ax[i//4, i%4].set_ylim([0, 1])
        ax[i//4, i%4].set_xlim([400, 700])

plt.tight_layout()
plt.show()


In [59]:
materials_matrix = np.array(data)
materials_matrix.shape

(370, 31)

## <font color='#4C5FDA'>**Calculamos los angulos espectrales para cada material con respecto a cada pixel en el cubo**</font>


In [60]:
import spectral

result_sam = spectral.algorithms.spectral_angles(None,None) 

In [61]:
result_sam.shape

(1392, 1300, 370)

In [62]:
def plot_2_signatures(signature1, signature2):
    plt.plot(wv_int, signature1, label='signature1')
    plt.plot(wv_int, signature2, label='signature2')
    
def plot_nearest_signature(nearest_matrix, x, y):
    material = nearest_matrix[x,y]
    row = spectraldb.iloc[material, :]
    signature = np.array([eval(row.SCEMeasures)[wavelength] for wavelength in wv_int]) / 100
    plot_2_signatures(cube[x,y,:], signature)
    plt.legend()
    plt.show()

In [63]:
import numpy as np

a = np.argmin(result_sam, axis=2)
a.shape

(1392, 1300)

In [ ]:
plot_nearest_signature(None,None,None)

In [65]:
segmentation_materials = np.zeros((cube.shape[0], cube.shape[1]), dtype=np.uint32)

for i in range(cube.shape[0]):
    for j in range(cube.shape[1]):
        material = a[i,j]
        row = spectraldb.iloc[material, :]

        material = str(row.Name)
        real_material =[category.lower() for category in materials if category.lower() in material.lower()]
        if len(real_material) == 0:
            real_material =[category.lower() for category in categories if category.lower() in row.ObjectType.lower()]

        segmentation_materials[i,j] = materials_dict[real_material[0]]
        #segmentation_materials[i,j] = objects[str(row.ObjectType)] #material


In [ ]:
materials_dict

In [ ]:
import matplotlib.patches as mpatches
import matplotlib

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

colors = ['olive','red', 'green', 'yellow', 'pink', 'orange', 'purple', 'brown', 'gray', 'cyan']
vmin = None
vmax = None
cmap = matplotlib.colors.ListedColormap(colors)


im = ax[1].imshow(segmentation_materials, cmap="tab20b")

values = list(range(11))
names = list(materials_dict.keys())

colors = [ im.cmap(im.norm(value)) for value in values]



aa = [ mpatches.Patch(color=colors[i], label=f"{list(materials_dict.keys())[list(materials_dict.values()).index(i+1)]}") for i in range(0,11) ]


plt.legend(handles=aa, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.2)

ax[0].imshow(img_rgb)

plt.show()

In [ ]:
materials_dict

In [ ]:
from skimage import data, segmentation, color

labels_slic = segmentation.slic(img_rgb, compactness=None, n_segments=None)
superpixels_slic = color.label2rgb(labels_slic, img_rgb, kind='avg')

# show original image, slic, ground truth
fig, ax = plt.subplots(1,3, figsize=(15,10))
ax[0].imshow(img_rgb)
ax[0].set_title("Original")
ax[0].axis('off')
ax[1].imshow(superpixels_slic)
ax[1].set_title("SLIC")
ax[1].axis('off')
ax[2].imshow(segmentation.mark_boundaries(img_rgb, labels_slic))

In [ ]:
def sp_idx(s, index = True):
     u = np.unique(s)
     return [np.where(s == i) for i in u], u.shape[0]

superpixel_list, n = sp_idx(labels_slic)

for i in range(n):
     superpixel = superpixel_list[i]
     objects_ = []
     for j in range(len(superpixel[0])):
          x = None
          y = None
          objects_.append(segmentation_materials[x,y])
     max_object = max(set(objects_), key=objects_.count)
     for j in range(len(superpixel[0])):
          x = None
          y = None
          segmentation_materials[x,y] = max_object

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
im = ax[1].imshow(segmentation_materials, cmap='tab20b')

values = list(range(13))
names = list(materials_dict.keys())
colors = [ im.cmap(im.norm(value)) for value in values]
aa = [ mpatches.Patch(color=colors[i], label=f"{list(materials_dict.keys())[list(materials_dict.values()).index(i+1)]}") for i in range(0,11) ]
plt.legend(handles=aa, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.2)

ax[0].imshow(img_rgb)

plt.show()

In [71]:
segmentation.shape

AttributeError: module 'skimage.segmentation' has no attribute 'shape'